In [1]:
import matplotlib.pyplot as plt # For plotting
import numpy as np              # Linear algebra library
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

from google.colab import files
uploaded = files.upload()
data = pd.read_csv("clean_dataset.csv")

Saving clean_dataset.csv to clean_dataset.csv


In [2]:
import re

expected_columns = ['id', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Label']

# Filter out rows that don't have all the expected columns
data = data.dropna(subset=expected_columns)


data = data.drop(['id'], axis=1)

categories = ["Siblings", "Co-worker", "Partner", "Friends"]
for category in categories:
    data[category] = data['Q5'].apply(lambda x: 1 if isinstance(x, str) and category in x.split(',') else 0)

# Drop the original 'Q5' column
data = data.drop(['Q5'], axis=1)

def extract_rankings(text):
    rankings = dict(item.split('=>') for item in text.split(','))
    return {k: int(v) if v else 0 for k, v in rankings.items()}

# Apply the function to the 'Q6' column and expand it into multiple columns
rankings_df = data['Q6'].apply(extract_rankings).apply(pd.Series)

# Concatenate the new columns with the original DataFrame, dropping the 'Q6' column
data = pd.concat([data.drop('Q6', axis=1), rankings_df], axis=1)

numeric_columns = ['Q7', 'Q8', 'Q9']
for col in numeric_columns:
    if data[col].dtype == object:  # Check if the column is of object (string) type
        data[col] = data[col].str.replace(',', '').astype(float)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


# Separate features and target variable
X = data.drop(['Q10', 'Label'], axis=1)  # Features  # Features
y = data['Label']  # Target variable

print('X:', X)



X:        Q1   Q2   Q3   Q4    Q7    Q8     Q9  Siblings  Co-worker  Partner  \
0     4.0  1.0  2.0  1.0  25.0   7.0  100.0         0          1        0   
1     4.0  3.0  5.0  2.0  20.0   3.0    4.0         0          1        0   
2     5.0  4.0  5.0  1.0  32.0   5.0    4.0         0          0        1   
3     5.0  4.0  4.0  1.0  23.0  10.0    3.0         0          0        1   
4     4.0  3.0  3.0  3.0  20.0  10.0    5.0         1          0        1   
...   ...  ...  ...  ...   ...   ...    ...       ...        ...      ...   
1463  4.0  2.0  5.0  3.0   5.0   2.0    2.0         1          0        1   
1464  5.0  3.0  4.0  2.0   7.0   2.0    4.0         0          1        1   
1465  2.0  1.0  3.0  4.0   9.0  87.0   67.0         0          0        0   
1466  5.0  4.0  5.0  3.0  15.0   1.0   15.0         0          0        1   
1467  4.0  3.0  5.0  4.0   5.0   5.0    5.0         0          0        1   

      Friends  Skyscrapers  Sport  Art and Music  Carnival  Cuisine  Eco

In [26]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# kNN model
knn = KNeighborsClassifier(n_neighbors=6)

# Train the model
knn.fit(X_train_scaled, y_train)

# Predict and evaluate the model
y_pred = knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy}')

Model Accuracy: 0.8309859154929577
